In [ ]:
# Define extra components of Model 2
def DefineModel2(Model):
    Model.PalletShelf = pyo.Var(Model.P, Model.S, within = pyo.NonNegativeIntegers, bounds = (0, Model.MaxShelfSize), initialize = Model.MaxShelfSize)

    Model.PalletFits.deactivate()   # This constraint is replaced by the following constraints

    # Bounds on ShelfHeights:
    # - Lower bound is zero, to allow for unused shelf positions. We retain the "0 *" part for completeness.
    # - Upper bound is maximum shelf size.
    
    def rule_fitlinear(Model, p):   # Each pallet must be allocated to a shelf that is at least the height of the pallet
        return sum(Model.PalletShelf[p, s] for s in Model.S) >= Model.Pallets[p]
    Model.PalletFitsLinear = pyo.Constraint(Model.P, rule = rule_fitlinear)

    def rule_fitLB1(Model, p, s):   # Linearization of fit constraint, part 1
        return 0 * Model.Allocation[p, s] <= Model.PalletShelf[p, s]
    Model.PalletFitsLB1 = pyo.Constraint(Model.P, Model.S, rule = rule_fitLB1)

    def rule_fitUB1(Model, p, s):   # Linearization of fit constraint, part 2
        return Model.MaxShelfSize * Model.Allocation[p, s] >= Model.PalletShelf[p, s]
    Model.PalletFitsUB1 = pyo.Constraint(Model.P, Model.S, rule = rule_fitUB1)

    def rule_fitUB2(Model, p, s):   # Linearization of fit constraint, part 3
        return Model.ShelfHeights[s] - Model.MaxShelfSize * (1 - Model.Allocation[p, s]) <= Model.PalletShelf[p, s]
    Model.PalletFitsUB2 = pyo.Constraint(Model.P, Model.S, rule = rule_fitUB2)
    
    def rule_fitLB2(Model, p, s):   # Linearization of fit constraint, part 4
        return Model.ShelfHeights[s] - 0 * (1 - Model.Allocation[p, s]) >= Model.PalletShelf[p, s]
    Model.PalletFitsLB2 = pyo.Constraint(Model.P, Model.S, rule = rule_fitLB2)